# Imports

In [ ]:
from functools import partial
from pathlib import Path

import h5py
import matplotlib.pyplot as plt
import nd2reader
import numpy as np
import zarr
from tqdm.auto import tqdm

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from paulssonlab.image_analysis import *
from paulssonlab.image_analysis.ui import display_image

# Reformat ND2 as Zarr/HDF5

In [ ]:
# TODO: need to write entire chunks at a time to avoid ZipFile duplicate file warning
def convert_nd2_to_zarr(nd2_filename, zarr_filename, slices):
    nd2 = nd2reader.ND2Reader(nd2_filename)
    frame = nd2.get_frame_2D()
    dtype = frame.dtype
    shape = frame.shape
    del frame
    channel_idxs = np.arange(nd2.sizes.get("c", 1))[slices.get("c", slice(None))]
    fovs = np.arange(nd2.sizes.get("v", 1))[slices.get("v", slice(None))]
    zs = np.arange(nd2.sizes.get("z", 1))[slices.get("z", slice(None))]
    ts = np.arange(nd2.sizes.get("t", 1))[slices.get("t", slice(None))]
    with zarr.ZipStore(zarr_filename, mode="w") as store:
        root = zarr.group(store=store)
        for channel_idx in tqdm(channel_idxs, desc="c"):
            channel = nd2.metadata["channels"][channel_idx]
            channel_group = root.create_group(channel)
            for fov in tqdm(fovs, desc="v"):
                ary = channel_group.zeros(
                    fov,
                    shape=(len(zs), len(ts), *shape),
                    chunks=(1, 5, None, None),
                    dtype=dtype,
                )
                for z_idx, z in enumerate(tqdm(zs, desc="z")):
                    for t_idx, t in enumerate(tqdm(ts, desc="t")):
                        ary[z_idx, t_idx, :, :] = nd2.get_frame_2D(
                            c=channel_idx, v=fov, z=z, t=t
                        )

In [ ]:
# TODO: json metadata in hdf5

In [ ]:
!ls /home/jqs1/scratch/jqs1/microscopy/230619/230619_NAO745_repressilators_split.nd2aa

In [ ]:
import glob

In [ ]:
filename = workflow.SplitFilename(
    glob.glob(
        "/home/jqs1/scratch/jqs1/microscopy/230619/230619_NAO745_repressilators_split.nd2*"
    )
)
# dest_filename = "/home/jqs1/scratch/jqs1/microscopy/230619/230619_NAO745_repressilators"

In [ ]:
filename

In [ ]:
nd2 = workflow.get_nd2_reader(filename)

In [ ]:
nd2.sizes

In [ ]:
display_image(nd2.get_frame_2D(t=0, v=259, c=0), downsample=2, scale=0.9)

In [ ]:
from paulssonlab.io.metadata import parse_nd2_metadata

In [ ]:
md = parse_nd2_metadata(nd2)

In [ ]:
multifile_filename = filename + ".full/"

In [ ]:
Path(multifile_filename).mkdir()

In [ ]:
!ls /home/jqs1/scratch/jqs1/microscopy/230619/230619_NAO745_repressilators_split*

In [ ]:
%%time
readers.convert_nd2_to_hdf5(
    nd2,
    multifile_filename,
    # file_axes=[],
    # dataset_axes=["fov", "t", "channel"],
    # slices=dict(fov=slice(22, 24), t=slice(3)),
)

# Send HDF5

In [ ]:
h5 = h5py.File(
    "/home/jqs1/scratch/jqs1/microscopy/230619/230619_NAO745_repressilators_split.2fov_3t/t=1_fov=22.hdf5",
)

In [ ]:
h5["channel=CFP-EM"][()].squeeze().shape

In [ ]:
display_image(h5["channel=CFP-EM"][()], downsample=2, scale=0.9)